In [2]:
%cd ..

c:\Users\catto\Desktop\Code\Project\TTS\StyleTTS2


In [3]:
# load packages
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

from models import *
from utils import *

device = 'cpu'#'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
#config = yaml.safe_load(open("Models/LibriTTS/config.yml"))
config = yaml.safe_load(open("Configs/config.yml"))
# ASR_path = config.get('ASR_path', False)
# text_aligner = ASRCNN(
#         input_dim = 80,
#         hidden_dim = 256,
#         n_token = 178,
#         token_embedding_dim = 512,
# )
# asr_params = torch.load(ASR_path, map_location='cpu')
# text_aligner.load_state_dict(asr_params['model'])

model_params = recursive_munch(config['model_params'])
#model = build_model(model_params, text_aligner = text_aligner, pitch_extractor = None)
model = build_model(model_params)

c:\Users\catto\anaconda3\Lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
c:\Users\catto\anaconda3\Lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [9]:
keys_to_keep = {'predictor', 'decoder', 'text_encoder','predictor_encoder', 'style_encoder', 'text_aligner', 'pitch_extractor', 'mpd', 'msd'} #remove 'bert', 'bert_encoder'; not needed in inference 'text_aligner'
#params_whole = torch.load("Models/LibriTTS/og.pth", map_location='cpu')
params_whole = torch.load("Models/Finetune_Removed/og_finetune.pth", map_location='cpu')
params = params_whole['net']
params = {key: value for key, value in params.items() if key in keys_to_keep}

for key in list(model.keys()):
    if key not in keys_to_keep:
        del model[key]

for key in model:
    if key in params:
        print('%s loaded' % key)
        try:
            model[key].load_state_dict(params[key])
        except:
            from collections import OrderedDict
            state_dict = params[key]
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k[7:] # remove `module.`
                new_state_dict[name] = v
            # load params
            model[key].load_state_dict(new_state_dict, strict=False)
#             except:
#                 _load(params[key], model[key])
_ = [model[key].eval() for key in model]
_ = [model[key].to(device) for key in model]

decoder loaded
predictor loaded
text_encoder loaded
predictor_encoder loaded
style_encoder loaded
text_aligner loaded
pitch_extractor loaded
mpd loaded
msd loaded


In [10]:
old_weight = [
    model['text_encoder'].embedding,
    model['text_aligner'].ctc_linear[2].linear_layer,
    model['text_aligner'].asr_s2s.embedding,
    model['text_aligner'].asr_s2s.project_to_n_symbols
]

In [11]:
for module in old_weight:
    print(module, module.weight.shape)

Embedding(178, 512) torch.Size([178, 512])
Linear(in_features=256, out_features=178, bias=True) torch.Size([178, 256])
Embedding(178, 512) torch.Size([178, 512])
Linear(in_features=128, out_features=178, bias=True) torch.Size([178, 128])


In [12]:
extend_to = 189

for i in range(len(old_weight)):
    new_shape = (extend_to, old_weight[i].weight.shape[1])
    new_weight = torch.randn(new_shape) * 0.01 #init mean=0, std=0.01
    with torch.no_grad():
        new_weight[:old_weight[i].weight.size(0), :] = old_weight[i].weight.detach().clone()
    new_param = nn.Parameter(new_weight, requires_grad=True)

    if isinstance(old_weight[i], nn.Embedding):
        old_weight[i].num_embeddings = extend_to
        
    if isinstance(old_weight[i], nn.Linear):
        old_weight[i].out_features = extend_to
        #update bias
        old_bias = old_weight[i].bias.detach()
        old_dim = old_bias.shape[0]
        new_bias = torch.zeros(extend_to)
        new_bias[:old_dim] = old_bias.clone()
        old_weight[i].bias.data = new_bias

    old_weight[i].weight = new_param

In [13]:
for module in old_weight:
    print(module, module.weight.shape)

Embedding(189, 512) torch.Size([189, 512])
Linear(in_features=256, out_features=189, bias=True) torch.Size([189, 256])
Embedding(189, 512) torch.Size([189, 512])
Linear(in_features=128, out_features=189, bias=True) torch.Size([189, 128])


In [14]:
save_path = "./Models/Finetune_Extend"
#save_path_asr = "./Utils/ASR"
if not os.path.exists(save_path):
    os.mkdir(save_path)
state = {
    'net':  {key: model[key].state_dict() for key in model}, 
    'optimizer': params_whole['optimizer'],
    'iters': params_whole['iters'],
    'val_loss': params_whole['val_loss'],
    'epoch': params_whole['epoch'],
}
save_path = osp.join(save_path, 'extended.pth')

# state_asr = {
#     'optimizer': asr_params['optimizer'],
#     'scheduler': asr_params['scheduler'],
#     'steps': asr_params['steps'],
#     'epochs': asr_params['epochs'],
#     'model': model['text_aligner'].state_dict(),
# }
# save_path_asr = osp.join(save_path_asr, 'extended_asr.pth')

torch.save(state, save_path)  
#torch.save(state_asr, save_path_asr)